In [1]:
import os
import json
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from shutil import copyfile

#Specify the image size that we are resizing to and the confidence level that we are filtering by
IMAGE_SIZE = 300  #Make change to this value to resize the image to a different size
CONFIDENCE_LEVEL = 0.4 #Make change to this value to filter by a different confidence level

#Specify the split ratio for the train and test sets, should sum up to 1
test_ratio = 0.3 #Make change to this value to change the ratio of the test set
train_ratio = 0.7 #Make change to this value to change the ratio of the train set


######################################################################################################################################################################

# Load the data
with open('image_recognition_file.json') as f: #the image_recognition_file.json file is the output from EcoAssist MegaDetector
    data = json.load(f)

# Ensure the necessary directories exist
os.makedirs('train', exist_ok=True)
os.makedirs('test', exist_ok=True)

def get_square_crop_coords(bbox, img_shape):
    x, y, w, h = bbox
    img_w, img_h = img_shape
    
    # Convert bbox from relative to absolute coordinates
    x = int(x * img_w)
    y = int(y * img_h)
    w = int(w * img_w)
    h = int(h * img_h)
    
    side_length = max(w, h)
    center_x = x + w // 2
    center_y = y + h // 2
    
    x1 = max(center_x - side_length // 2, 0)
    y1 = max(center_y - side_length // 2, 0)
    x2 = min(center_x + side_length // 2, img_w)
    y2 = min(center_y + side_length // 2, img_h)
    
    # Adjust to ensure square dimensions
    side_length = min(x2 - x1, y2 - y1)
    return x1, y1, x1 + side_length, y1 + side_length

# List to hold file paths and their respective species
file_paths = []
species_labels = []

# Process images
for img_data in tqdm(data['images']):
    file_path = img_data['file']
    species = file_path.split('/')[0]
    img = Image.open(file_path)
    
    if 'detections' in img_data:
        for i, detection in enumerate(img_data['detections']):
            if detection['category'] == "1" and detection['conf'] >= CONFIDENCE_LEVEL: # Only include images with a confidence of 0.4 or higher and a category of 1(animal)
                bbox = detection['bbox']
                x1, y1, x2, y2 = get_square_crop_coords(bbox, img.size)
                cropped_img = img.crop((x1, y1, x2, y2))
                resized_img = cropped_img.resize((IMAGE_SIZE, IMAGE_SIZE))
                
                # Convert RGBA to RGB if necessary
                if resized_img.mode == 'RGBA':
                    resized_img = resized_img.convert('RGB')
                
                # Ensure directories exist
                os.makedirs(f'train/{species}', exist_ok=True)
                os.makedirs(f'test/{species}', exist_ok=True)
                
                # Save resized image to temporary path with a unique name
                temp_path = f'temp/{species}_{os.path.basename(file_path).split(".")[0]}_{i}.jpg'
                os.makedirs('temp', exist_ok=True)
                resized_img.save(temp_path)
                
                file_paths.append(temp_path)
                species_labels.append(species)

# Split data into train and test sets
train_files, test_files, train_labels, test_labels = train_test_split(file_paths, species_labels, test_size=test_ratio, stratify=species_labels)

# Save images to respective directories
for file_list, labels, dir_name in zip([train_files, test_files], [train_labels, test_labels], ['train', 'test']):
    for i, f in enumerate(file_list):
        species = labels[i]
        output_path = f'{dir_name}/{species}/{os.path.basename(f)}'
        copyfile(f, output_path)

print("Processing complete!")


100%|██████████| 145/145 [00:02<00:00, 62.22it/s]


Processing complete!
